In [ ]:
import quandl
quandl.ApiConfig.api_key = ''

In [ ]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
tickers = ['SPY','TLT','TIP','GLD']
factors = ['S&P 500','Treasury Bonds','TIPS','Gold']
fund_df = quandl.get_table('SHARADAR/SFP', ticker=tickers, paginate=True)
fund_df.sort_values(by='date', ascending=True, inplace=True)

In [ ]:
master = fund_df[fund_df['ticker']=='SPY'][['date','close']].copy(deep=True)
master.set_index('date', inplace=True)
master.rename({'close': factors[0]}, axis=1, inplace=True)

div = fund_df[fund_df['ticker']=='SPY'][['date','dividends']].copy(deep=True)
div.set_index('date', inplace=True)
div.rename({'dividends': factors[0]}, axis=1, inplace=True)

for i, ticker in enumerate(tickers[1:]):
    merge_df = fund_df[fund_df['ticker']==ticker][['date','close']].set_index('date')
    master = master.merge(merge_df, how='left', left_index=True, right_index=True)
    master.rename({'close': factors[i+1]}, axis=1, inplace=True)
    
    merge_df = fund_df[fund_df['ticker']==ticker][['date','dividends']].set_index('date')
    div = div.merge(merge_df, how='left', left_index=True, right_index=True)
    div.rename({'dividends': factors[i+1]}, axis=1, inplace=True)
    
master.dropna(inplace=True)

final = master/master.shift(1)-1 + div/master

In [ ]:
print(final.tail(5))

In [ ]:
# final is a dataframe of daily returns for the assets

# I use the historical mean return for my expected return
E = np.array(final.mean(axis=0)).reshape(-1,1)

# Calculate the covariance matrix of the asset's returns
cov_matrix = np.array(final.cov())

# Ones vector
ones = np.ones((E.shape[0],1))

zeros = np.zeros((2,2))

In [ ]:
# Put together the A matrix
A = 2*cov_matrix
A = np.append(A, E.T, axis=0)
A = np.append(A, ones.T, axis=0)
temp = np.append(E, ones, axis=1)
temp = np.append(temp, zeros, axis=0)
A = np.append(A, temp, axis=1)

# Put together the b vector
b = np.array([[0],
              [0],
              [0],
              [0],
              E[0],  # I set the target return to be
              [1]])  # the expected return of stocks

# So in essense, I am looking for an optimal portfolio
# that is expected to give the same return as I get from
# investing in stocks (but with lower risk)

In [ ]:
# Optimize using matrix algebra

results = inv(A)@b
# Grab first 4 elements (because 4 assets)
opt_W = results[:final.shape[1]]

print(pd.DataFrame(opt_W, index=final.columns, columns=['Optimal Weights']))

In [ ]:
# Portfolio expected return
opt_W.T@E*250

In [ ]:
# Portfolio volatility
(opt_W.T@cov_matrix@opt_W)**0.5*(250**0.5)

In [ ]:
backtest = pd.concat([final, final@opt_W], axis=1)
backtest.rename({0: 'Optimal'}, axis=1, inplace=True)
ax = ((backtest+1).cumprod()-1).plot(kind='line', figsize=(9,6));
ax.set_ylabel('Cumulative Return')
plt.tight_layout()
plt.savefig('optimized_portfolio', bpi=150)

In [ ]:
print(pd.DataFrame(A))

In [ ]:
print(pd.DataFrame(b))